In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# getting the rating/budget data
total_df = pd.read_csv('./data/imdb_ratings_finance.csv')
total_df = total_df.loc[total_df["budget"].notna()]
total_df = total_df.loc[total_df["budget"] != np.float64(0.0)]
total_df = total_df.loc[total_df["global_revenue"].notna()]
total_df = total_df.loc[total_df["global_revenue"] != np.float64(0.0)]
total_df = total_df.rename(columns={"global_revenue":"revenue"})
total_df = total_df.drop(columns={"release_date"})
total_df = total_df.set_index("title")
total_df

,runtime,genre,averageRating,budget,revenue
title,,,,,
Star Wars Ep. VII: The Force Awakens,NaN,NaN,NaN,533200000.0,2.056047e+09
Avatar: The Way of Water,192,"Action,Adventure,Fantasy",7.5,460000000.0,2.317514e+09
Indiana Jones and the Dial of Destiny,154,"Action,Adventure,Sci-Fi",6.5,402300000.0,3.839631e+08
Avengers: Endgame,181,"Action,Adventure,Drama",8.4,400000000.0,2.748243e+09
Pirates of the Caribbean: On Stranger Tides,136,"Action,Adventure,Fantasy",6.6,379000000.0,1.045714e+09
...,...,...,...,...,...
Cavite,80,"Crime,Drama,Thriller",6.0,7000.0,7.164400e+04
The Mongol King,NaN,NaN,NaN,7000.0,9.000000e+02
Following,69,"Crime,Mystery,Thriller",7.4,6000.0,2.404950e+05


In [3]:
# importing horror info
horror_df = pd.read_excel("./data/horror_movies.xlsx")
horror_df = horror_df[["title", "original_language", "budget", "revenue", "status", "release_date"]]
# we removed the overview but it may be useful later
horror_df = horror_df.loc[horror_df["status"] != "In Production", :]
horror_df['release_year'] = horror_df['release_date'].dt.year
horror_df['release_month'] = horror_df['release_date'].dt.month
horror_df = horror_df.drop(columns={"release_date", "status", "budget", "revenue"})
horror_df = horror_df.set_index("title")
horror_df

,original_language,release_year,release_month
title,,,
Oracle,en,2022,12
Puppeteer,ru,2022,12
The Field,en,2022,12
Scare Package II: Rad Chad’s Revenge,en,2022,12
Unicorn Wars,es,2022,12
...,...,...,...
Kankal,bn,1950,7
The Fall of the House of Usher,en,1950,6
The Rockingham Tea Set,en,1950,1


In [35]:
combined_df = horror_df.merge(total_df, left_index=True, right_index=True)
combined_df = combined_df[~combined_df.index.duplicated(keep='first')]
combined_df

,original_language,release_year,release_month,runtime,genre,averageRating,budget,revenue
title,,,,,,,,
Venus,es,2022,12,\N,\N,5.9,6000000.0,7818479.0
The Menu,en,2022,11,90,"Crime,Drama",6.2,30000000.0,75820378.0
Terror Train,en,2022,10,97,"Horror,Mystery,Thriller",5.8,3500000.0,8000000.0
Halloween Ends,en,2022,10,111,"Drama,Horror,Mystery",5.0,20000000.0,105400796.0
The Visitor,en,2022,10,108,"Horror,Sci-Fi",5.2,4000000.0,19174817.0
...,...,...,...,...,...,...,...,...
A Ghost Story,en,1964,2,92,"Drama,Fantasy,Mystery",6.8,100000.0,2769782.0
Insomnia,fr,1963,1,85,Comedy,5.5,46000000.0,113622499.0
Where the Truth Lies,fr,1962,3,83,Thriller,6.4,25000000.0,1415656.0


In [37]:
# adjusting for inflation
inflation_df = pd.read_csv("./data/inflation.csv")
inflation_df["CPI_Multiplier"] = inflation_df["CPIAUCNS"].iloc[-1] / inflation_df["CPIAUCNS"]
inflation_df = inflation_df[["DATE","CPI_Multiplier"]]
inflation_df["DATE"] = pd.to_datetime(inflation_df["DATE"], format="%Y-%m-%d")

combined_df["release_day"] = 1

new_df = combined_df[['release_year', 'release_month', 'release_day']].rename(columns={"release_year":"year","release_month":"month","release_day":"day"})
combined_df['infl_date'] = pd.to_datetime(new_df)

combined_index = combined_df.index
combined_df = pd.merge(combined_df, inflation_df, how='left', left_on='infl_date', right_on="DATE").set_index(combined_index)
combined_df["real_budget"] = combined_df["budget"] * combined_df["CPI_Multiplier"] 
combined_df["real_revenue"] = combined_df["revenue"] * combined_df["revenue"] 
combined_df = combined_df.drop(columns=["budget", "revenue", "release_day", "infl_date", "DATE", "CPI_Multiplier"]).rename(columns={"real_budget":"budget", "real_revenue":"revenue"})
combined_df

,original_language,release_year,release_month,runtime,genre,averageRating,budget,revenue
title,,,,,,,,
Venus,es,2022,12,\N,\N,5.9,6.374074e+06,6.112861e+13
The Menu,en,2022,11,90,"Crime,Drama",6.2,3.177252e+07,5.748730e+15
Terror Train,en,2022,10,97,"Horror,Mystery,Thriller",5.8,3.703051e+06,6.400000e+13
Halloween Ends,en,2022,10,111,"Drama,Horror,Mystery",5.0,2.116029e+07,1.110933e+16
The Visitor,en,2022,10,108,"Horror,Sci-Fi",5.2,4.232058e+06,3.676736e+14
...,...,...,...,...,...,...,...,...
A Ghost Story,en,1964,2,92,"Drama,Fantasy,Mystery",6.8,1.020392e+06,7.671692e+12
Insomnia,fr,1963,1,85,Comedy,5.5,4.771002e+08,1.291007e+16
Where the Truth Lies,fr,1962,3,83,Thriller,6.4,2.618779e+08,2.004082e+12


In [38]:
# creating profit column
profit_col = []
norm_revenue_col = []
for i in combined_df.index:
    profit_col.append(combined_df.loc[i, "revenue"] - combined_df.loc[i, "budget"])
profit_col = pd.DataFrame({"agg_profit":profit_col})
profit_col = profit_col.set_index(combined_df.index)
combined_df = pd.concat([combined_df, profit_col], axis=1)
combined_df

,original_language,release_year,release_month,runtime,genre,averageRating,budget,revenue,agg_profit
title,,,,,,,,,
Venus,es,2022,12,\N,\N,5.9,6.374074e+06,6.112861e+13,6.112861e+13
The Menu,en,2022,11,90,"Crime,Drama",6.2,3.177252e+07,5.748730e+15,5.748730e+15
Terror Train,en,2022,10,97,"Horror,Mystery,Thriller",5.8,3.703051e+06,6.400000e+13,6.400000e+13
Halloween Ends,en,2022,10,111,"Drama,Horror,Mystery",5.0,2.116029e+07,1.110933e+16,1.110933e+16
The Visitor,en,2022,10,108,"Horror,Sci-Fi",5.2,4.232058e+06,3.676736e+14,3.676736e+14
...,...,...,...,...,...,...,...,...,...
A Ghost Story,en,1964,2,92,"Drama,Fantasy,Mystery",6.8,1.020392e+06,7.671692e+12,7.671691e+12
Insomnia,fr,1963,1,85,Comedy,5.5,4.771002e+08,1.291007e+16,1.291007e+16
Where the Truth Lies,fr,1962,3,83,Thriller,6.4,2.618779e+08,2.004082e+12,2.003820e+12


In [39]:
# adding a normalized revenue column
norm_revenue_col = []
for i in combined_df.index:
    norm_rev = int(combined_df.loc[i, "revenue"]) / (2023 - (combined_df.loc[i, "release_year"]))
    norm_revenue_col.append(norm_rev)
norm_revenue_col = pd.DataFrame({"norm_revenue":norm_revenue_col})
norm_revenue_col = norm_revenue_col.set_index(combined_df.index)
combined_df = pd.concat([combined_df, norm_revenue_col], axis=1)
combined_df

,original_language,release_year,release_month,runtime,genre,averageRating,budget,revenue,agg_profit,norm_revenue
title,,,,,,,,,,
Venus,es,2022,12,\N,\N,5.9,6.374074e+06,6.112861e+13,6.112861e+13,6.112861e+13
The Menu,en,2022,11,90,"Crime,Drama",6.2,3.177252e+07,5.748730e+15,5.748730e+15,5.748730e+15
Terror Train,en,2022,10,97,"Horror,Mystery,Thriller",5.8,3.703051e+06,6.400000e+13,6.400000e+13,6.400000e+13
Halloween Ends,en,2022,10,111,"Drama,Horror,Mystery",5.0,2.116029e+07,1.110933e+16,1.110933e+16,1.110933e+16
The Visitor,en,2022,10,108,"Horror,Sci-Fi",5.2,4.232058e+06,3.676736e+14,3.676736e+14,3.676736e+14
...,...,...,...,...,...,...,...,...,...,...
A Ghost Story,en,1964,2,92,"Drama,Fantasy,Mystery",6.8,1.020392e+06,7.671692e+12,7.671691e+12,1.300287e+11
Insomnia,fr,1963,1,85,Comedy,5.5,4.771002e+08,1.291007e+16,1.291007e+16,2.151679e+14
Where the Truth Lies,fr,1962,3,83,Thriller,6.4,2.618779e+08,2.004082e+12,2.003820e+12,3.285380e+10


In [40]:
combined_df.to_csv("processed.csv")